In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
import os
import shutil

In [2]:
df2 = pd.read_csv('cxr-record-list.csv')
df4 = pd.read_csv('mimic-cxr-2.0.0-chexpert.csv')
df6 = pd.read_csv('mimic-cxr-2.0.0-metadata.csv')
merge_df = df2.merge(df4,on=['subject_id','study_id'],how='inner')
merge_df = merge_df.merge(df6[['dicom_id', 'ViewPosition']],on='dicom_id',how='inner')

In [3]:
def csv_to_train_val(df,img_path,disease):
    '''
    split data to train ans validation
    disease labe must be 1 for case or 0 for control, labeled by chexpert
    images are all posterior to anterior (filtered when downloading)
    choose one PA image per subject
    '''
    file_ids = [file.replace(".jpg","") for file in os.listdir(img_path)]
    df = df[df['dicom_id'].isin(file_ids)]
    
    df.loc[:,'image_path'] = img_path+ '/' + df['dicom_id'] + '.jpg'
    
    input_df = df[df[disease].isin([0,1])].drop_duplicates('subject_id')
    train_df, val_df = train_test_split(
        input_df, 
        test_size=0.2, 
        random_state=1, 
        stratify=input_df[disease]
    )
    return train_df, val_df

def csv_to_test(df,img_path,disease):
    '''
    disease labe must be 1 for case or 0 for control, labeled by radiologist
    choose one PA image per subject
    '''
    file_ids = [file.replace(".jpg","") for file in os.listdir(f'{img_path}/{disease}')]
    df = df[df['dicom_id'].isin(file_ids)]
    
    df.loc[:,'image_path'] = img_path+ f'/{disease}/' + df['dicom_id'] + '.jpg'
    test_df = df[df[disease].isin([0,1])].drop_duplicates('subject_id')

    return test_df

In [4]:
# 1. Define the custom dataset class
class disease_Dataset(Dataset):
    def __init__(self, df, disease, transform=None):
        self.df = df
        self.transform = transform
        self.disease = disease
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['image_path']).convert("RGB") #images are gray scale, but models require RGB
        label = torch.tensor(row[self.disease], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label


# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def generate_dataLoader(disease, train_df,val_df, test_df):
    # DataLoaders
    train_dataset = disease_Dataset(train_df,disease, transform)
    val_dataset = disease_Dataset(val_df,disease, transform)
    test_dataset = disease_Dataset(test_df,disease, transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    test_loader = DataLoader(test_dataset, batch_size=32)
    return train_loader, val_loader, test_loader

In [5]:
def train(model, train_loader, val_loader, lr=1e-5, weight_decay=1e-4, dropout_prob=0.5, use_scheduler=True):

    # Add dropout to the fully connected layer dynamically
    model.fc = torch.nn.Sequential(
        torch.nn.Dropout(p=dropout_prob),
        torch.nn.Linear(model.fc.in_features, 1)
    ).to(device)  # Ensure the modified layer is on the correct device

    # Define loss function and optimizer
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Optionally use a learning rate scheduler
    scheduler = None
    if use_scheduler:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.1, patience=2, verbose=True
        )

    # Training loop with early stopping
    num_epochs = 20
    patience = 3  # Number of epochs to wait for improvement
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            # Move images and labels to the specified device
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_train_loss = running_loss / len(train_loader.dataset)

        # Evaluate on the validation set
        model.eval()
        val_labels = []
        val_preds = []
        val_running_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                # Move images and labels to the specified device
                images, labels = images.to(device), labels.to(device)
                outputs = model(images).squeeze()
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * images.size(0)
                preds = torch.sigmoid(outputs).cpu().numpy()  # Move predictions to CPU for evaluation
                val_preds.extend(preds)
                val_labels.extend(labels.cpu().numpy())  # Move labels to CPU for evaluation

        epoch_val_loss = val_running_loss / len(val_loader.dataset)

        # Validation metrics
        val_binary_preds = [1 if p > 0.5 else 0 for p in val_preds]
        val_accuracy = accuracy_score(val_labels, val_binary_preds)
        val_f1 = f1_score(val_labels, val_binary_preds)
        val_auc = roc_auc_score(val_labels, val_preds)

        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Training Loss: {epoch_train_loss:.4f}")
        print(f"  Validation Loss: {epoch_val_loss:.4f}")
        print(f"  Validation Accuracy: {val_accuracy:.4f}")
        print(f"  Validation F1 Score: {val_f1:.4f}")
        print(f"  Validation AUC: {val_auc:.4f}")

        # Update the learning rate scheduler
        if use_scheduler:
            scheduler.step(epoch_val_loss)

        # Check for early stopping
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            patience_counter = 0  # Reset counter if validation loss improves
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered. Stopping training.")
                break

    return model

In [6]:
def test(model, test_loader, disease):
    model.eval()
    test_labels, test_preds = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze()

            # Ensure outputs are always a batch
            if outputs.dim() == 0:  # If outputs is scalar, make it a batch
                outputs = outputs.unsqueeze(0)

            preds = torch.sigmoid(outputs).cpu().numpy()
            test_preds.extend(preds)
            test_labels.extend(labels.cpu().numpy())
    
    test_binary_preds = [1 if p > 0.5 else 0 for p in test_preds]
    test_accuracy = accuracy_score(test_labels, test_binary_preds)
    test_f1 = f1_score(test_labels, test_binary_preds)
    test_auc = roc_auc_score(test_labels, test_preds)

    print("\nTest Evaluation:")
    print(f"  Test Accuracy: {test_accuracy:.4f}")
    print(f"  Test F1 Score: {test_f1:.4f}")
    print(f"  Test AUC: {test_auc:.4f}")


In [9]:
from torchvision.models import resnet18, ResNet18_Weights

predict_models = {}
for disease in ['Pneumonia', 'Pleural Effusion', 'Pneumothorax', 'Atelectasis',
                'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
                'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding',
                'Pleural Other', 'Support Devices']:
    # Use .copy() to avoid SettingWithCopyWarning
    train_df, val_df = csv_to_train_val(merge_df.copy(), 'images_NEW', disease)
    test_df = csv_to_test(merge_df.copy(), 'Testing', disease)
    
    train_loader, val_loader, test_loader = generate_dataLoader(disease, train_df, val_df, test_df)

    # Updated ResNet18 to use weights
    weights = ResNet18_Weights.DEFAULT
    model = resnet18(weights=weights)
    model.fc = torch.nn.Linear(model.fc.in_features, 1)
    model = model.to(device)

    print(f"=====training {disease}=====")
    try:
        # Ensure the train function does not use verbose=True
        model = train(model, train_loader, val_loader, lr=1e-5, weight_decay=1e-4, dropout_prob=0.5, use_scheduler=True)
        predict_models[disease] = model
        print(f"=====testing {disease}=====")

        test(model, test_loader, disease)
        torch.save(model.state_dict(), f"CNN_models/{disease}_model_weights.pth")
    except ValueError:
        print(f"ValueError {disease}")


/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/hsieh.fe/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to

=====training Pneumonia=====
Epoch 1/20:
  Training Loss: 0.7871
  Validation Loss: 0.7530
  Validation Accuracy: 0.4034
  Validation F1 Score: 0.4668
  Validation AUC: 0.5552
Epoch 2/20:
  Training Loss: 0.7244
  Validation Loss: 0.6606
  Validation Accuracy: 0.5945
  Validation F1 Score: 0.4159
  Validation AUC: 0.6090
Epoch 3/20:
  Training Loss: 0.6839
  Validation Loss: 0.6201
  Validation Accuracy: 0.6858
  Validation F1 Score: 0.4599
  Validation AUC: 0.6655
Epoch 4/20:
  Training Loss: 0.6580
  Validation Loss: 0.6177
  Validation Accuracy: 0.6582
  Validation F1 Score: 0.3534
  Validation AUC: 0.6317
Epoch 5/20:
  Training Loss: 0.6372
  Validation Loss: 0.5889
  Validation Accuracy: 0.6985
  Validation F1 Score: 0.3302
  Validation AUC: 0.6633
Epoch 6/20:
  Training Loss: 0.6073
  Validation Loss: 0.5818
  Validation Accuracy: 0.7006
  Validation F1 Score: 0.3380
  Validation AUC: 0.6647
Epoch 7/20:
  Training Loss: 0.5905
  Validation Loss: 0.5664
  Validation Accuracy: 0.71

/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy o

=====training Pleural Effusion=====


/home/hsieh.fe/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


...

Epoch 1/20:
  Training Loss: 1.2682
  Validation Loss: 1.0867
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0000
  Validation AUC: 0.8000
Epoch 2/20:
  Training Loss: 1.3249
  Validation Loss: 1.1126
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0000
  Validation AUC: 0.3000
Epoch 3/20:
  Training Loss: 1.1927
  Validation Loss: 1.1716
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0000
  Validation AUC: 0.4000
Epoch 4/20:
  Training Loss: 1.4403
  Validation Loss: 0.9447
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0000
  Validation AUC: 0.4000
Epoch 5/20:
  Training Loss: 1.2193
  Validation Loss: 1.0572
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0000
  Validation AUC: 0.3000
Epoch 6/20:
  Training Loss: 1.1931
  Validation Loss: 1.0310
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0000
  Validation AUC: 0.6000
Epoch 7/20:
  Training Loss: 1.2831
  Validation Loss: 1.0569
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0

/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy o

=====training Support Devices=====


/home/hsieh.fe/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20:
  Training Loss: 0.7549
  Validation Loss: 0.6618
  Validation Accuracy: 0.6226
  Validation F1 Score: 0.7500
  Validation AUC: 0.6125
Epoch 2/20:
  Training Loss: 0.7144
  Validation Loss: 0.6376
  Validation Accuracy: 0.6792
  Validation F1 Score: 0.8046
  Validation AUC: 0.4833
Epoch 3/20:
  Training Loss: 0.7616
  Validation Loss: 0.6199
  Validation Accuracy: 0.6981
  Validation F1 Score: 0.8222
  Validation AUC: 0.5458
Epoch 4/20:
  Training Loss: 0.6932
  Validation Loss: 0.5571
  Validation Accuracy: 0.8302
  Validation F1 Score: 0.9032
  Validation AUC: 0.6500
Epoch 5/20:
  Training Loss: 0.6566
  Validation Loss: 0.5843
  Validation Accuracy: 0.7170
  Validation F1 Score: 0.8352
  Validation AUC: 0.5042
Epoch 6/20:
  Training Loss: 0.6451
  Validation Loss: 0.6066
  Validation Accuracy: 0.7170
  Validation F1 Score: 0.8352
  Validation AUC: 0.4042
Epoch 7/20:
  Training Loss: 0.6316
  Validation Loss: 0.6160
  Validation Accuracy: 0.6792
  Validation F1 Score: 0.8

In [62]:
# model = torch.load("model.pth")
# model = model.to(device)  # Move to GPU/CPU as needed
# model.eval()  # Set the model to evaluation mode
